In [22]:
import numpy as np 
import pandas as pd 
import os
from transformers import BertPreTrainedModel, BertModel, BertTokenizerFast, AdamW
from tqdm import tqdm

In [3]:
train = pd.read_csv("train_val_combined.tsv", sep="\t") 

train = train.loc[:50,:]

In [4]:
train.shape

(51, 3)

In [7]:
def split_text(s, overlap=20, chunk_size=180): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0: 
        n = len(s.split()) // (chunk_size - overlap)  
    else: 
        n = 1 
    for w in range(n):  
        if w == 0:  
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else: 
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size] 
            total.append(" ".join(partial)) 
    return total

In [14]:
queries = train['queries'].values 
positives = train['positive'].values 
negatives = train['negative'].values

In [18]:
text_file = open("original_text.txt", "w") 
text = queries[10] 
text_file.write(text) 
text_file.close()

In [20]:
text_file = open("after_sliding_window.txt", "w") 
text = queries[10] 
splitted = split_text(text) 

for t in splitted: 
    text_file.write(t + "\n\n") 

text_file.close() 

In [24]:
df_q, df_pos, df_neg = [], [], [] 

for i in tqdm(range(train.shape[0])): 
    q, pos, neg = queries[i], positives[i], negatives[i] 
    q_splits = split_text(q) 
    pos_splits = split_text(pos) 
    neg_splits = split_text(neg) 
    
    # get all combinations 
    for j in range(len(q_splits)): 
        for k in range(len(pos_splits)): 
            for l in range(len(neg_splits)): 
                df_q.append(q_splits[j]) 
                df_pos.append(pos_splits[k]) 
                df_neg.append(neg_splits[l]) 

100%|██████████| 51/51 [00:00<00:00, 222.10it/s]


In [26]:
df = pd.DataFrame(list(zip(df_q, df_pos, df_neg)),
               columns =['queries', 'positive', 'negative'])


df.head()

,queries,positive,negative
0,The invention using the modified hump contoure...,A fishing lure comprising an elongated body ha...,An integrated circuit comprising a ferroelectr...
1,The invention using the modified hump contoure...,A fishing lure comprising an elongated body ha...,oxide layer and the ferroelectric memory cell ...
2,The invention using the modified hump contoure...,A fishing lure comprising an elongated body ha...,and second oxide layers have different thickne...
3,The invention using the modified hump contoure...,A fishing lure comprising an elongated body ha...,wherein the at least one additive is provided ...
4,The invention using the modified hump contoure...,A fishing lure comprising an elongated body ha...,layer system comprising at least one bi layer ...


In [29]:
df.to_excel('splitted_train_sample.xlsx', index=False)

In [30]:
df.to_csv("splitted_train_sample.tsv", index=False, sep='\t')